In [1]:
import pandas as pd
import numpy as np
import math
import time

## 数据读取

In [2]:
data_home = "./"

In [3]:
triplet_dataset = pd.read_csv(filepath_or_buffer = data_home +"output.csv", 
                              sep=",", header=None, 
                              names=["user_id","goods_id","click_count","add_cart_count","order_count"])

/Users/wjj/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
triplet_dataset.shape

(21500149, 5)

In [5]:
triplet_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21500149 entries, 0 to 21500148
Data columns (total 5 columns):
user_id           object
goods_id          object
click_count       int64
add_cart_count    int64
order_count       int64
dtypes: int64(3), object(2)
memory usage: 820.2+ MB


In [26]:
#对每一个用户，分别统计他的click总量
out_dict = {}
with open(data_home + "output.csv") as f:
    #enumerate给出可遍历的对象的索引和索引值
    for line_number, line in enumerate(f):
        user = line.split(",")[0]
        click_count = int(line.split(",")[2]) + int(line.split(",")[3]) * 5 + int(line.split(",")[4]) * 10
       
        #如果用户以前就有过点击行为，累加
        if user in out_dict:
            click_count += out_dict[user]
            out_dict.update({user:click_count})
        #如果用户第一次统计点击行为，赋值
        out_dict.update({user:click_count})
output_list = [{"user":k,"click_count":v} for k,v in out_dict.items()]

In [27]:
click_count_df = pd.DataFrame(output_list)
click_count_df.isnull().any()

click_count    False
user           False
dtype: bool

In [28]:
#以点击数量为指标排序，方便过滤掉点击量太少的用户
click_count_df = click_count_df.sort_values(by = "click_count", ascending=False)

In [30]:
click_count_df.shape

(4619238, 2)

In [31]:
click_count_df.to_csv(path_or_buf="user_click_count_df.csv", index = False)

In [32]:
click_count_df.isnull().any()

click_count    False
user           False
dtype: bool

In [35]:
#没意义了
#取其中一部分数（按大小排好序的了，这些应该是比较重要的数据），作为我们的实验数据
total_click_count = sum(click_count_df.click_count)
#取前30万个用户拿出来看看，发现点击数已经占到总数的40%，说明很多用户都是惰性用户
#如果一个用户点击量太少，比如1，那么他对这个商品打分为1，这不合适，因为他只是随便看看。必须要点击量达到一定数量的统计才有可信度。
print(float(click_count_df.head(n=400000).click_count.sum()) / total_click_count)
#460多万个用户里前40万用户点击数占比76.2%

0.7625129925335955


In [38]:
print(float(click_count_df.head(n=300000).click_count.sum()) / total_click_count)
#前30万用户点击数占比72%

0.7205310828630093


In [67]:
triplet_dataset_sum_df = pd.merge(triplet_dataset_sub,click_count_df, how="inner", left_on="user_id", right_on="user")

In [76]:
triplet_dataset_sum_df.rename(columns={"click_count_y":"click_count_sum"},inplace = True)

In [77]:
triplet_dataset_sum_df.rename(columns={"click_count_x":"click_count_goods"},inplace = True)

In [81]:
del(triplet_dataset_sum_df["user"])

In [95]:
triplet_dataset_sum_df["click_count_sum_sqrt"] = math.sqrt(triplet_dataset_sum_df["click_count_sum"][2:])

TypeError: cannot convert the series to <class 'float'>

In [135]:
for i in triplet_dataset_sum_df["click_count_sum"]:
    triplet_dataset_sum_df["click_count_sum_sqrt"] = round(math.sqrt(i), 3)
    

KeyboardInterrupt: 

In [109]:
#计算商品被当前用户点击量 / 用户点击总量 当做用户对这个商品的打分值
triplet_dataset_sum_df["score"] = round(((triplet_dataset_sum_df["click_count_goods"] + triplet_dataset_sum_df["add_cart_count"]*5 + triplet_dataset_sum_df["order_count"]*10) / triplet_dataset_sum_df["click_count_sum"])*100, 3)

In [ ]:
#计算商品被当前用户点击量 / 用户点击总量的平方 以对活跃用户对打分进行补偿
triplet_dataset_sum_df["score"] = ((triplet_dataset_sum_df["click_count_goods"] + triplet_dataset_sum_df["add_cart_count"]*5 + triplet_dataset_sum_df["order_count"]*10) / triplet_dataset_sum_df["click_count_sum_sqrt"])*100

In [111]:
triplet_dataset_sum_df.isnull().any()

user_id                 False
goods_id                False
click_count_goods       False
add_cart_count          False
order_count             False
click_count_sum         False
score                   False
click_count_sum_sqrt    False
dtype: bool

In [112]:
triplet_dataset_sum_df.to_csv(path_or_buf=data_home+"triplet_dataset_sum_df.csv", index=False)

## 开始计算item相似度

In [114]:
dict_user = {}
for line in open(data_home+"triplet_dataset_sum_df.csv", "r", encoding="utf-8"):
    #根据head（）来改动
    user_id,goods_id,click_count_goods, add_cart_count, order_count, click_count_sum, score, click_count_sum_sqrt= line.strip().split(',')
    #itemSim.setdefault(goods_id, 0)
    dict_user.setdefault(str(user_id), {})    
    dict_user[user_id][goods_id] = score

In [115]:
del dict_user['user_id']

In [137]:
class ItemCF:
    def __init__(self):
        self.user_score_dict = dict_user
        # self.items_sim = self.ItemSimilarity()
        self.items_sim = self.ItemSimilarityBest()

    #  计算item之间的相似度
    def ItemSimilarity(self):
        itemSim = {}
        # 得到每个物品有多少用户产生过行为
        item_user_count = {}
        # 共现矩阵，表示同时喜欢i 和j两个items的用户数
        count = {}
        for user, item in self.user_score_dict.items():
            #i相当于item。
            for i in item.keys():
                item_user_count.setdefault(i, 0)
                #如果该用户对该商品评分大于0
                if self.user_score_dict[user][i] > 0.0:
                    item_user_count[i] += 1
                for j in item.keys():
                    count.setdefault(i, {}).setdefault(j, 0)
                    if (
                        self.user_score_dict[user][i] > 0.0
                        and self.user_score_dict[user][j] > 0.0
                        and i != j
                    ):
                        count[i][j] += 1
        # 共现矩阵 -> 相似度矩阵
        for i, related_items in count.items():
            itemSim.setdefault(i, {})
            for j, cuv in related_items.items():
                itemSim[i].setdefault(j, 0)
                itemSim[i][j] = cuv / item_user_count[i]
        return itemSim

    # 计算item之间的相似度 优化：进行了热门商品的惩罚
    def ItemSimilarityBest(self):
        itemSim = {}
        # 得到每个物品有多少用户产生过行为
        item_user_count = {}
        # 共现矩阵
        count = {}
        for user, item in self.user_score_dict.items():
            for i in item.keys():
                item_user_count.setdefault(i, 0)
                if float(self.user_score_dict[user][i]) > 0.0:
                    item_user_count[i] += 1
                for j in item.keys():
                    count.setdefault(i, {}).setdefault(j, 0)
                    #print(self.user_score_dict[user][i])
                    #print("*")
                    #print(self.user_score_dict[user])
                    #print("-")
                    if (
                        float(self.user_score_dict[user][i]) > 0.0
                        and float(self.user_score_dict[user][j]) > 0.0
                        and i != j
                    ):
                        count[i][j] += 1
        # 共现矩阵 -> 相似度矩阵
        for i, related_items in count.items():
            itemSim.setdefault(i, {})
            for j, cuv in related_items.items():
                itemSim[i].setdefault(j, 0)
                itemSim[i][j] = cuv / math.sqrt(item_user_count[i] * item_user_count[j])
        return itemSim

    # 预测用户对item的评分
    def preUserItemScore(self, userA, item):
        score = 0.0
        #print(self.items_sim[item])
        for item1 in self.items_sim[item].keys():
            try:
                if item1 != item:
                    score += (
                    self.items_sim[item][item1] * float(self.user_score_dict[userA][item])
                    )
            
            except Exception as e:
                
                continue            
        return score

    # 为用户推荐物品
    def recommend(self, userA):
        # 计算userA 未评分item的可能评分
        user_item_score_dict = {}
        for item in self.user_score_dict[userA].keys():
            # if self.user_score_dict[userA][item] <= 0:
            user_item_score_dict[item] = self.preUserItemScore(userA, item)
        return user_item_score_dict

{'p_easyfun_qdszh_190726': 13.616051168931124, 'p_ea_ximeiyin_210ml': 2.630124937939902, 'p_easyfun_zishuzhou_700g': 13.145536588272948, 'easyfun_jzbc_130g_180604': 9.881863819468267, 's_easyfun_niurougan_250g_180808': 16.191853603624118, 'p_cm25_csysj_qm_28g_180330': 9.005942418661679, 's_tm25_yy_meiguo_420g': 14.628980033724497, 's_cm25_green_coffee_140ml': 15.76409231822356, 'p_bu_BH20SW_white': 17.935266827336967, 's_tm25_yyy_x_43_bh_170119': 16.870921146246406, 's_easyfun_zishuzhou_350g': 19.361035917861656, 'p_easyfun_qdszh_zishuzou_qiyazi_dgkf': 11.091827640608328, 's_bhsx_roller_170712': 7.000270770667846, 'p_tm25_mx_330_6_190424': 7.899933868996712, 'p_easyfun_geshidongka_100g': 13.285572577717119, 'p_easyfun_fqjd_4he_181106': 7.713361739380919, 's_ef_ysalb_160g': 14.244786813171565, 's_oday_muscleball_3color': 7.8592823814617105, 's_oday_dizhijixiongrou_100g': 19.588077261822583, 'p_tm25_yykk_xc_new_190103': 7.6685890240205765, 's_bhydg_tlqtld': 7.510488074636171, 's_oday_jix

In [ ]:
ib = ItemCF()
    